Intended to be viewed as a slide show using RISE.

In [1]:
from IPython.display import YouTubeVideo

In [ ]:
!docker rmi sillyex:0
!docker rmi sillyex:1
!docker rmi sillyex:2

Intended to be viewed as a slide show using RISE.

----

<h1 style="font-family:Helvetica;color:#007399;">Docker,</h1>
### Just enough to get your feet wet

-----

<h1 style="font-family:Helvetica;color:#007399;">Intro</h1>

# Scope:



+ A super **brief** and **basic** intro. *- Will skip over most **info**, as it is covered **everywhere** else.*


+ A quick look at how to use it to run **postgresql** localy.


+ An example of how I like to use it. `[if time]`


+ What you can expect if you decided to put your **whole** dev enviroment in a container. `[if time]`


\**Notebook will be made available, I will also include links to info that I skipped.*





## Basically...

I will try to get you up and running with postgres on your local `[Linux]` machine using docker, and, time willing,
show some other, hopefully motivational, examples.

+ Focus will be on Linux, becuase:
  - Using it on OSX requries more steps than I have time to cover. It's not hard, just requires a lot more explaining.
  - I've never run Docker on Windows. With the realease of Docker-Machine it's probable similar to running it on OSX.

\**Docker-Macine is out of scope.*

----

<h1 style="font-family:Helvetica;color:#007399;">Basics</h1>

<center><img src="http://gifsec.com/wp-content/uploads/GIF/2014/09/Monkey-and-Banana-GIF.gif?gs=a" style="width: 500px; display:inline"/></center>
<p style="text-align:center; margin-top:50px; display:inline-block">Less time dealing with DBA's and OP's.</p>

<p style="text-align:center">More time playing with tech.</p>

<center><img src="http://cdn.list25.com/wp-content/uploads/2012/04/monkey.gif" style="width: 500px;"/></center>

<p style="text-align:center">Also, Docker images are super portable. You can take them and run them anywhere.</p>

<center><img src="http://ak-hdl.buzzfed.com/static/enhanced/web05/2012/8/8/19/anigif_enhanced-buzz-23480-1344469198-5.gif" style="width: 500px;"/></center>

\**End of monkey gifs*

----

## Containers Vs Images

**An Image is:**
 - Kinda like a Class

**A Container is:**
 - Kinda like an instance of a Class
 
 - Temporary, disposable

![](img/docker-filesystems-multilayer.png)

## List your Images

**All Images:** `docker images`

In [2]:
!docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             VIRTUAL SIZE
etlcore             base                865639df6186        34 hours ago        656.4 MB
textquant           run                 445d392b1524        34 hours ago        5.861 GB
textquant           base                6ca70e8565c4        34 hours ago        5.837 GB
etlcore             py2                 54bd9d38f9bd        34 hours ago        938 MB
ellis_island        run                 69ee44237956        4 days ago          1.535 GB
ellis_island        base                ec7286be5372        4 days ago          1.138 GB
postgres_test       latest              278c88bb7ef0        4 weeks ago         361.6 MB
postgres            latest              a7d662bede59        6 weeks ago         265.3 MB
ubuntu              15.04               013f3d01d247        7 weeks ago         131.4 MB
ubuntu              14.04.3             91e54dfb1179        7 weeks ago         188.4 MB
ubuntu             

## List your Containers

**All running containers: **
`docker ps`

In [3]:
!docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
8f2d2fefabdb        postgres_test       "/usr/lib/postgresql/"   4 weeks ago         Up 34 hours         0.0.0.0:2345->5432/tcp   pg_test


**All containers:** `docker ps -a`

In [4]:
!docker ps -a

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
8f2d2fefabdb        postgres_test       "/usr/lib/postgresql/"   4 weeks ago         Up 34 hours         0.0.0.0:2345->5432/tcp   pg_test


-------

<h1 style="font-family:Helvetica;color:#007399;">Anantomy of a Docker Build Directory</h1>

```
textproc-base
  ├── Dockerfile
  ├── nlkt-data.tar
  ├── requirements.txt
  └── stanford-ner-2015-04-20.zip
```

In [5]:
!cd /Users/steven_c/Docker/textquant-docker/ && tree textquant_base

textquant_base
├── Dockerfile
├── nlkt_data.tar
├── requirements.txt
└── stanford-ner-2015-04-20.zip

0 directories, 4 files


### Docker Files

Contain instructions for building an image.

Example Commands

```bash
RUN apt-get -y supertuxkart

ENV PATH $PATH:/some/bin/

COPY requirements.txt /tmp/

RUN pip install -r /tmp/requirements.txt
```

### Example Dockerfile

```bash
FROM etlcore:py2
MAINTAINER Steven Cutting <steven.e.cutting@linux.com>

RUN apt-get update 

RUN apt-get install -y -qq default-jdk

# --
# Stanford NER
COPY stanford-ner-2015-04-20.zip /usr/local/bin/
RUN cd /usr/local/bin/ &&  unzip stanford-ner-2015-04-20.zip

ENV STANFORD_MODELS :/usr/local/bin/stanford-ner-2015-04-20
ENV CLASSPATH $CLASSPATH:/usr/local/bin/stanford-ner-2015-04-20/stanford-ner.jar
ENV STANFORD_MODELS $STANFORD_MODELS:/usr/local/bin/stanford-ner-2015-04-20/classifiers/english.all.3class.distsim.crf.ser.gz
ENV STANFORD_MODELS $STANFORD_MODELS:/usr/local/bin/stanford-ner-2015-04-20/classifiers/english.conll.4class.distsim.crf.ser.gz
ENV STANFORD_MODELS $STANFORD_MODELS:/usr/local/bin/stanford-ner-2015-04-20/classifiers/english.muc.7class.distsim.crf.ser.gz
ENV STANFORD_MODELS $STANFORD_MODELS:/usr/local/bin/stanford-ner-2015-04-20/classifiers/example.serialized.ncc.ncc.ser.gz

# --
# Python packages
COPY requirements.txt /tmp/
RUN pip install -r /tmp/requirements.txt

# --
# data files
COPY nltk_data.tar /tmp/
RUN tar -xpf /tmp/nltk_data.tar -C /usr/share && chown -R root:users /usr/share/nltk_data/*
```

<h1 style="font-family:Helvetica;color:#007399;">A Silly Example</h1>

That will hopefully give you a more concrete idea of how Docker Images work.

#### Create a simple Docker Image:

-  Builds on top of the Ubuntu 14.04 base image
-  Creates an enviroment variable (env) within the image.
-  Prints the value of the env to the screen **during** the build processes (not when running).

In [7]:
!rm -r silly_ex 
!mkdir silly_ex && mkdir silly_ex/silly_ex_0

DOCKERFILE = """

FROM ubuntu:14.04

ENV TESTLIST monkey_one
RUN echo $TESTLIST

"""
with open('silly_ex/silly_ex_0/Dockerfile', 'w') as fp:
    fp.write(DOCKERFILE)

Building the **Image** and giving it the **name:** `sillyex` and the **tag:** `0`.

In [8]:
!cd silly_ex/silly_ex_0 && docker build --tag="sillyex:0" .

Sending build context to Docker daemon 2.048 kB
Step 0 : FROM ubuntu:14.04
 ---> 91e54dfb1179
Step 1 : ENV TESTLIST monkey_one
 ---> Running in 95e37eae452c
 ---> cb3d6db51d3d
Removing intermediate container 95e37eae452c
Step 2 : RUN echo $TESTLIST
 ---> Running in 3d6d924d5792
monkey_one
 ---> fe03643cc6b6
Removing intermediate container 3d6d924d5792
Successfully built fe03643cc6b6


In [9]:
!mkdir silly_ex/silly_ex_1

DOCKERFILE = """

FROM sillyex:0
MAINTAINER Steven Cutting <steven.e.cutting@linux.com>

ENV TESTLIST $TESTLIST:monkey_two
RUN echo $TESTLIST

"""
with open('silly_ex/silly_ex_1/Dockerfile', 'w') as fp:
    fp.write(DOCKERFILE)

Building the **Image** and giving it the **name:** `sillyex` and the **tag:** `1`.

In [10]:
!cd silly_ex/silly_ex_1 && docker build --tag="sillyex:1" .

Sending build context to Docker daemon 2.048 kB
Step 0 : FROM sillyex:0
 ---> fe03643cc6b6
Step 1 : MAINTAINER Steven Cutting <steven.e.cutting@linux.com>
 ---> Running in 0f53e713d285
 ---> 7a5f1f717c7c
Removing intermediate container 0f53e713d285
Step 2 : ENV TESTLIST $TESTLIST:monkey_two
 ---> Running in 7f6462a628f6
 ---> a7a4b598301e
Removing intermediate container 7f6462a628f6
Step 3 : RUN echo $TESTLIST
 ---> Running in b12ef4dd8d83
monkey_one:monkey_two
 ---> 700dc2cc7512
Removing intermediate container b12ef4dd8d83
Successfully built 700dc2cc7512


In [11]:
!mkdir silly_ex/silly_ex_2

DOCKERFILE = """

FROM sillyex:1
MAINTAINER Steven Cutting <steven.e.cutting@linux.com>

ENV TESTLIST $TESTLIST:monkey_three
RUN echo $TESTLIST

# A little hack... Just go with it.
RUN echo "echo $TESTLIST" > test.sh && chmod +x test.sh
CMD ["/bin/bash", "./test.sh"]
"""
with open('silly_ex/silly_ex_2/Dockerfile', 'w') as fp:
    fp.write(DOCKERFILE)

Building the **Image** and giving it the **name:** `sillyex` and the **tag:** `2`.

In [12]:
!cd silly_ex/silly_ex_2 && docker build --tag="sillyex:2" .

Sending build context to Docker daemon 2.048 kB
Step 0 : FROM sillyex:1
 ---> 700dc2cc7512
Step 1 : MAINTAINER Steven Cutting <steven.e.cutting@linux.com>
 ---> Running in d1a2f5b533d3
 ---> f531e989aacd
Removing intermediate container d1a2f5b533d3
Step 2 : ENV TESTLIST $TESTLIST:monkey_three
 ---> Running in 7534ea5434e9
 ---> 2e674a6eaef1
Removing intermediate container 7534ea5434e9
Step 3 : RUN echo $TESTLIST
 ---> Running in 8a94c742f32e
monkey_one:monkey_two:monkey_three
 ---> 60be3c421e05
Removing intermediate container 8a94c742f32e
Step 4 : RUN echo "echo $TESTLIST" > test.sh && chmod +x test.sh
 ---> Running in bc9465d88eb6
 ---> a0552967870a
Removing intermediate container bc9465d88eb6
Step 5 : CMD /bin/bash ./test.sh
 ---> Running in a1ad4c37768d
 ---> 35bfc13160d9
Removing intermediate container a1ad4c37768d
Successfully built 35bfc13160d9


### Lets try runnig the image

In [13]:
!docker run sillyex:2

monkey_one:monkey_two:monkey_three


### If I look at the Images and all Containers:

In [14]:
!docker ps -a

CONTAINER ID        IMAGE               COMMAND                  CREATED                  STATUS                              PORTS                    NAMES
377efc248cdf        sillyex:2           "/bin/bash ./test.sh"    Less than a second ago   Exited (0) Less than a second ago                            suspicious_davinci
8f2d2fefabdb        postgres_test       "/usr/lib/postgresql/"   4 weeks ago              Up 34 hours                         0.0.0.0:2345->5432/tcp   pg_test


In [15]:
!docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             VIRTUAL SIZE
sillyex             2                   35bfc13160d9        1 seconds ago       188.4 MB
sillyex             1                   700dc2cc7512        3 seconds ago       188.4 MB
sillyex             0                   fe03643cc6b6        3 seconds ago       188.4 MB
etlcore             base                865639df6186        34 hours ago        656.4 MB
textquant           run                 445d392b1524        34 hours ago        5.861 GB
textquant           base                6ca70e8565c4        34 hours ago        5.837 GB
etlcore             py2                 54bd9d38f9bd        34 hours ago        938 MB
ellis_island        run                 69ee44237956        4 days ago          1.535 GB
ellis_island        base                ec7286be5372        4 days ago          1.138 GB
postgres_test       latest              278c88bb7ef0        4 weeks ago         361.6 MB
postgres           

```
sillyex:0
└── sillyex:1
    └── sillyex:2
```

<img  style="width:300px; float:left;" src="https://image.freepik.com/free-photo/cut-onion-vector_2935315.jpg" />

-----

<h1 style="font-family:Helvetica;color:#007399;">How do you actually "Use" this, Docker?</h1>

-----

**<p style="font-size:1.8em">Docker is like a Meeseeks Box and Containers are like Meeseeks.</p>**

<p style="font-size: 1.3em">"How?" you might ask.</p>
<p>Or really, "Whuuuuuuuu the heck is a Meeseeks?"</p>

-----

<p style="font-size:1.4em">I'm glad you asked!</p>
**<p style="font-size:1.8em">This short clip should hopefully explain.</p>**
<p style="font-size:1em">(Yes a clip, just humor me.)</p>

In [3]:
YouTubeVideo("qUYvIAP3qQk")

Because of this it you can start to think differnetly about the way to use/distribute applications and infrastructure.

Because Docker is so lightweight you can use it to really seperate your dev enviroment by containerizing the heck out of eveything. You can even Docker your Docker...

### Docker can start to seem like Inception.

<center><img src="http://myownspot.org/wordpress/julie/wp-content/uploads/2014/01/Inception-Turning-the-City-2.gif" style="width: 1000px;" /></center>

<p style="font-size: 0.7em;"><strong>*Disclaimer:</strong> I haven't actually seen Inception, I just want to sound smart.</p>

<p style="font-size: 0.7em;">And, this is a pretty neat gif.</p>

# Ex:

I'm running all of this inside a docker container. 

I'm running docker inside docker.

Inception!

<img src="https://media.giphy.com/media/6y17pgEsBdN7y/giphy.gif" />